# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [2]:
import pandas as pd
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [3]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [4]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [5]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [6]:
document_root = ET.parse( './data/mondial_database.xml' )

In [38]:
# 1. 10 countries with the lowest infant mortality rates

# convert xml to pandas dataframe for easier processing
def parse_countries_to_df(countries):
    for country in countries:
        country_dict = {}
        country_dict['car_code'] = country.attrib['car_code']
        country_dict['name'] = country.find('name').text
        if country.find('infant_mortality') is not None:
            country_dict['infant_mortality'] = country.find('infant_mortality').text
        
        yield country_dict
        
countries = document_root.findall('country')
countries_df = pd.DataFrame(list(parse_countries_to_df(countries)))

countries_df

,car_code,infant_mortality,name
0,AL,13.19,Albania
1,GR,4.78,Greece
2,MK,7.9,Macedonia
3,SRB,6.16,Serbia
4,MNE,NaN,Montenegro
5,KOS,NaN,Kosovo
6,AND,3.69,Andorra
7,F,3.31,France
8,E,3.33,Spain
9,A,4.16,Austria


In [8]:
# convert infant_mortality column to floats
countries_df['infant_mortality'] = countries_df['infant_mortality'].apply(pd.to_numeric)

# sort
sorted_countries = countries_df.sort_values('infant_mortality')

# result
sorted_countries[['name','infant_mortality']].head(10)

,name,infant_mortality
38,Monaco,1.81
98,Japan,2.13
117,Bermuda,2.48
36,Norway,2.48
106,Singapore,2.53
37,Sweden,2.60
10,Czech Republic,2.63
78,Hong Kong,2.73
79,Macao,3.13
44,Iceland,3.15


In [9]:
def get_item_latest_population(item): # item can be either city or country
    populations = item.findall('population')
    populationsCount = len(populations) 
        
    if populationsCount > 0:
        return populations[populationsCount - 1].text
    else:
        return 0

In [10]:
def parse_cities_to_df(cities):
    for city in cities:
        city_dict = {}
        city_dict['name'] = city.find('name').text
        city_dict['population'] = get_item_latest_population(city)
        yield city_dict

In [11]:
# 10 cities with the largest population
cities = document_root.findall('country/city')
cities_df = pd.DataFrame(list(parse_cities_to_df(cities)))

cities_df

,name,population
0,Tirana,418495
1,Shkodër,77075
2,Durrës,113249
3,Vlorë,79513
4,Elbasan,78703
5,Korçë,51152
6,Skopje,514967
7,Kumanovo,107745
8,Beograd,1639121
9,Novi Sad,335701


In [12]:
# convert population column to floats
cities_df['population'] = cities_df['population'].apply(pd.to_numeric)

# sort
sorted_cities = cities_df.sort_values('population', ascending=False)

# result
sorted_cities[['name','population']].head(10)

,name,population
176,Seoul,9708483
164,Al Qahirah,8471859
80,Bangkok,7506700
128,Hong Kong,7055071
92,Ho Chi Minh,5968384
212,Singapore,5076700
163,Al Iskandariyah,4123869
216,New Taipei,3939305
177,Busan,3403135
107,Pyongyang,3255288


In [13]:
# 3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [14]:
def get_group_percentage(group):
    return float(group.attrib['percentage'])

In [15]:
def get_group_population(group, country_population):
    return (get_group_percentage(group) / 100) * country_population

In [16]:
def create_group(group, country):
    country_population = float(get_item_latest_population(country))
    
    group_dict = {}
    group_dict['group'] = group.text
    group_dict['country'] = country.find('name').text
    group_dict['population'] = get_group_population(group, country_population)
    return group_dict

In [17]:
def get_country_groups(country):
    groups_list = []
    
    groups = country.findall('ethnicgroup')
    
    if len(groups) > 0:
        for group in groups:
            groups_list.append(create_group(group, country))
    
    return groups_list

In [18]:
def get_countries_groups(countries):
    groups = []
    for country in countries:
        groups += get_country_groups(country)
    
    return groups

In [19]:
countries = document_root.findall('country')
groups_df = pd.DataFrame(list(get_countries_groups(countries)))
groups_df

,country,group,population
0,Albania,Albanian,2.660131e+06
1,Albania,Greek,8.400414e+04
2,Greece,Greek,1.005915e+07
3,Macedonia,Macedonian,1.322388e+06
4,Macedonia,Albanian,5.190681e+05
5,Macedonia,Turkish,8.033197e+04
6,Macedonia,Gypsy,5.561444e+04
7,Macedonia,Serb,3.707629e+04
8,Serbia,Serb,5.903032e+06
9,Serbia,Montenegrin,6.408599e+04


In [20]:
# sum up the same ethnic groups in different countries
group_totals = groups_df.groupby('group').sum()

# sort 
sorted_group_totals = group_totals.sort_values('population', ascending=False)

# result
sorted_group_totals.head(10)

,population
group,
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.948722e+08
African,3.183251e+08
Dravidian,3.027137e+08
Mestizo,1.577344e+08
Bengali,1.467769e+08
Russian,1.318570e+08
Japanese,1.265342e+08


In [21]:
#  4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [ ]:
# a) longest river

In [32]:
def parse_rivers_to_df(rivers):
    for river in rivers:
        river_dict = {}
        river_dict['name'] = river.find('name').text
        river_dict['country'] = river.attrib['country']
        if river.find('length') is not None:
            river_dict['length'] = river.find('length').text
        
        yield river_dict

In [40]:
# create a dataframe
rivers = document_root.findall('river')
rivers_df = pd.DataFrame(list(parse_rivers_to_df(rivers)))

In [47]:
longest_river = rivers_df.sort_values('length', ascending=False)
longest_river_country = longest_river['country']
longest_river_country

121             R MNG
86                MNE
219               ZRE
215           ANG ZRE
169               MEX
65           RO MD UA
98                 IR
57     SLO HR BIH SRB
4                   S
18             NL B F
31                  E
204               CAM
212               ZRE
28                E P
82            D PL CZ
217               ZRE
236               AUS
21               F CH
129           UZB KGZ
104              R EW
25                  F
108             BY UA
85           PL BY UA
35                  I
54       I A SLO HR H
29                E P
194               SSD
99                  R
102                 R
97                 TR
            ...      
216               ZRE
221               ZRE
232               RSA
155               CDN
15                 SF
152           CDN USA
60            MNE BIH
149           CDN USA
131           AFG TAD
209               ZRE
191           SUD ETH
207       ZRE RCA RCB
183         RG WAG SN
214           ANG ZRE
72        